## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
path_to_dir = '/content/drive/MyDrive/Datasets'
path_to_file = "/GGMarquez/CADS.txt"





### Importamos

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time

In [ ]:
tf.keras.utils.text_dataset_from_directory(
    path_to_dir,
    labels="inferred",
    label_mode="int",
    class_names=None,
    batch_size=32,
    max_length=None,
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    follow_links=False,
)

### Veamos el texto


In [ ]:
# Leer y decodear.
text = open(path_to_dir + path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')

In [ ]:
print(text[:218])

In [ ]:
# Los caracteres (alfabeticos, comas, puntos, espacios...)
vocab = sorted(set(text))
print(f'{len(vocab)} caracteres')

## Procesamos el texto

### Vectorizamos
Hay que pasar strings a una codificacion numérica

Con `tf.keras.layers.StringLookup` se puede hacer, hay que tokenizar el texto antes

In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

# Qué es un ragged tensor?

Creamos `tf.keras.layers.StringLookup`

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

De token a char_id

In [ ]:
ids = ids_from_chars(chars)
ids

Para recobrar desde el id se puede usar `tf.keras.layers.StringLookup(..., invert=True)`.  

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
chars = chars_from_ids(ids)
chars

Y reducimos

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)




# La predicción


Para un caracter concreto, cuál es más probable que le siga? El input será una secuencia de caracteres, y entrenamos el modelo para la predicción del siguiente

Al ser RNNs, mantienen un estado interno que depende de lo visto anteriormente


### Crearemos ejemplos y targets
Dividiremos el texto en secuencias, de longitud `seq_length` caracteres.

Para cada secuencia de input , los targets serán del mismo tamaño pero empezando un caracter más tarde.

Así que partimos el texto. Digamos que  `seq_length` es 3 y nuestro texto "Hola". el input será "Hol", y el ouptut "ola".

Con `tf.data.Dataset.from_tensor_slices` pasamos de vector de texto a vector de indices

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

In [ ]:
seq_length = 100


Con batch lo podemos hacer cómodamente

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

#Por qué seq_length +1?

It's easier to see what this is doing if you join the tokens back into strings:

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

Para entrenar, hay que crear un conjunto `(input, label)`.

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(3):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

### Creamos los Batches para entrenamiento



In [ ]:
# Batch size
BATCH_SIZE = 190

# Buffer es para que Tensorflow nos reserve esa memoria
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

## Al modelo!

Usamos un modelo de 3 capas

* `tf.keras.layers.Embedding`: El input;
* `tf.keras.layers.GRU`: Un tipo de RNN (también se podrá hacer con LSTMs)
* `tf.keras.layers.Dense`: El output. Cuantas neuronas habrá aquí? Y qué representan?

In [ ]:
#Definimos los tamaños de nuestras capas

vocab_size = len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self) #super() es por las herencias
    # definimos las capas
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)


  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x) #Qué créeis que hace esto?
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training) #Y esto?

    if return_state:
      return x, states #Y esto?
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)



![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

## Veamos el modelo

Primero el output:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [ ]:
# Y esto qué forma tiene?
model.summary()

A ver qué tal predice

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [ ]:
sampled_indices

Decode these to see the text predicted by this untrained model:

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Más bien regular

## A entrenar

Usamos crossentropy (a grandes rasgos, cuánta información se necesita para explicar la diferencia)

Como el modelo trabaja con logits, hay que avisar a la funcion de pérdida

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

In [ ]:
tf.exp(example_batch_mean_loss).numpy() #A qué se parece este número

Es el tamaño del vocabulario

In [ ]:
model.compile(optimizer='adam', loss=loss)

###Configuramos los checkpoints

Para que no se pierdan durante el entrenamiento

In [ ]:
# Directorio a guardar
checkpoint_dir = './training_checkpoints_GGMarquez'
# Nombre
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")



checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=False,
    verbose = 1,
     save_freq= 600)





Ejecutamos el entrenamiento

In [ ]:
EPOCHS = 20

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback],verbose= 2)

## Generando texto

Le estamos pasando cada vez que hace una predicción un input y un valor para su estado interno, y cuando ejecute devolverá su output y un estado actualizado

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=.3):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Creamos una mascara para evitar problemas con unicode
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Manejamos índices erróneos
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Encajamos los tamaños. Qué hace esto?
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function #esto es un decorador de tensorflow
  def generate_one_step(self, inputs, states=None):
    # strings a IDS
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Ejecutamos
    # predicted_logits.shape es [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # solo la última predicción
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # aplicamos la máscara parea no generar UNKs
    predicted_logits = predicted_logits + self.prediction_mask

    # Generamos tokenIDS
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Token id a caracteres
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Devuelvo las predicciones y estados
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Si lo ejecutamos en bucle, generará strings caracter a caracter

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['azada'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

O en paralelo

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Aureliano', 'estirpe', 'hielo', 'Aureliano', 'Buendia'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

## Exportamos


In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states = None
next_char = tf.constant(['Muchos años despues, frente al peloton de fusilamiento'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))